In [29]:
import os
import numpy
import matplotlib.pyplot as pyplot
from itertools import groupby 
import random
import shutil
%matplotlib inline

In [48]:
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

Using TensorFlow backend.


In [46]:
def structureData(source_data, train_perc):
    images = []

    for (dirpath, dirnames, filenames) in os.walk(source_data):
        images.extend(filenames)

    data_path = './data'
    if not os.path.exists(data_path):
        os.makedirs(data_path)

    util_func = lambda x: x[0] 
    temp = sorted(images, key = util_func) 
    res = [list(ele) for i, ele in groupby(temp, util_func)]

    train_path = os.path.join(data_path, 'train')
    os.makedirs(train_path)
    test_path = os.path.join(data_path, 'test')
    os.makedirs(test_path)

    for i in range(len(res)):
        perc = 0.7
        z_0 = res[i]
        train_len = int(len(z_0) * 0.7)
        random.shuffle(z_0)
        train_p = z_0[:train_len]
        test_p = z_0[train_len:]
        tr_path = os.path.join(train_path, str(i))
        os.makedirs(tr_path)
        te_path = os.path.join(test_path, str(i))
        os.makedirs(te_path)
        for name in train_p:
            shutil.copy(os.path.join(source_data, name), tr_path)
        for name in test_p:
            shutil.copy(os.path.join(source_data, name), te_path)

In [47]:
structureData('./dataset', 0.7)

In [95]:
datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        rotation_range=10,
        horizontal_flip=True,
        vertical_flip=False)


In [96]:
img_shape = (28, 28)

In [97]:
train_generator = datagen.flow_from_directory(
    'data/train',
    target_size=img_shape,
    color_mode = 'grayscale',
    batch_size=32,
    class_mode='categorical')

Found 236 images belonging to 7 classes.


In [98]:
test_generator = datagen.flow_from_directory(
    'data/test',
    target_size= img_shape,
    color_mode = 'grayscale',
    batch_size=32,
    class_mode='categorical')

Found 105 images belonging to 7 classes.


In [99]:
input_shape = (img_shape[0], img_shape[1], 1)

In [100]:
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = input_shape))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(units = 7 , activation = 'softmax'))
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 28, 28, 75)        750       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 75)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 14, 14, 50)        33800     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 7, 7, 50)          0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 7, 7, 25)          11275     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 4, 4, 25)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 400)              

In [101]:
model.fit_generator(
    train_generator,
    steps_per_epoch=32,
    epochs=20,
    validation_data=test_generator)

Epoch 1/20


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_13/convolution (defined at /home/adam-kowalczyk/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_7701]

Function call stack:
keras_scratch_graph
